In [1]:
import os
import shutil
import pandas as pd
from sqlalchemy import create_engine

from dotenv import load_dotenv
from dotenv import find_dotenv
load_dotenv(find_dotenv())

engine = create_engine(os.environ.get('SQLALCHEMY_DATABASE_URI'), echo=True)

In [2]:
nome_relatorio = 'congl_financeiros_resumo'
file_name = '{}.csv'.format(nome_relatorio)
df = pd.read_csv(os.path.join('bases', file_name), low_memory=False)
df.columns
df.head()

# remove unnamed columns
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Unnamed: 19', axis=1, inplace=True)
df.drop('Unnamed: 20', axis=1, inplace=True)

a_renomear = {
    'Instituição financeira':'nome_if',
    'Código':'co_if',
    'TCB':'tp_consolidado_bancario',
    'SR':'segmento',
    'TD':'tp_consolidacao',
    'TC':'tp_controle',
    'Cidade':'cidade',
    'UF':'uf',
    'Data':'dt_base',
    'Ativo Total':'ativo_total',
    'Carteira de Crédito Classificada':'carteira_credito_classificada',
    'Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros':'passivo_circulante_exigivel_longo_prazo',
    'Captações':'captacoes',
    'Patrimônio Líquido':'patrimonio_liquido',
    'Lucro Líquido':'lucro_liquido',
    'Patrimônio de Referência para Comparação com o RWA':'patrimonio_referencia',
    'Índice de Basileia':'ic_basileia',
    'Índice de Imobilização':'ic_imobilizacao',
    'Número de Agências':'nu_agencias',
    'Número de Postos de Atendimento':'nu_postos_atendimento',
    'Conglomerado':'conglomerado',
    'Conglomerado Financeiro':'conglomerado_financeiro',
    'Conglomerado Prudencial':'conglomerado_prudencial',
    'TI':'ti'
}

# renomeia as colunas
df = df.rename(columns=a_renomear)

# remove informações que são consolidadas
df = df[df['co_if'].notnull()]

# convert just columns "a" and "b"
df['co_if'] = df['co_if'].astype(int)
df['tp_controle'] = df['tp_controle'].astype(int)

print(df.head())
print(df.dtypes)

# salva os registros no banco de dados
df.to_sql('{}_import'.format(nome_relatorio), con=engine, if_exists='replace')

# executa para ver os resultados retornados que foram importados
df_banco = engine.execute("SELECT * FROM {}_import".format(nome_relatorio)).fetchall()
print(len(df_banco))

print('Registros importados com sucesso.')

                                             nome_if     co_if  \
0                                                 BB     49906   
1                            CAIXA ECONOMICA FEDERAL    360305   
2  BANCO NACIONAL DE DESENVOLVIMENTO ECONOMICO E ...  33657248   
3                                           BRADESCO     10045   
4                                               ITAU     10069   

  tp_consolidado_bancario segmento tp_consolidacao  tp_controle  \
0                      b1      NaN               C            1   
1                      b1      NaN               I            1   
2                      b4      NaN               I            1   
3                      b1      NaN               C            2   
4                      b1      NaN               C            2   

           cidade  uf  dt_base  ativo_total  ... nu_agencias  \
0        BRASILIA  DF  03/2000  126.009.362  ...       2.903   
1        BRASILIA  DF  03/2000  124.494.699  ...       1.702   
2  RIO D

2020-02-12 11:45:02,692 INFO sqlalchemy.engine.base.Engine COMMIT
2020-02-12 11:45:02,734 INFO sqlalchemy.engine.base.Engine SELECT * FROM congl_financeiros_resumo_import
2020-02-12 11:45:02,736 INFO sqlalchemy.engine.base.Engine {}
144198
Registros importados com sucesso.


In [ ]:
nome_relatorio = 'congl_financeiros_ativo'
file_name = '{}.csv'.format(nome_relatorio)
df = pd.read_csv(os.path.join('bases', file_name), low_memory=False)
print(df.columns)

# remove unnamed columns
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Unnamed: 15', axis=1, inplace=True)
df.drop('Unnamed: 20', axis=1, inplace=True)
df.drop('Unnamed: 21', axis=1, inplace=True)
df.drop('Unnamed: 26', axis=1, inplace=True)
df.drop('Unnamed: 28', axis=1, inplace=True)

a_renomear = {
    'Instituição financeira':'nome_if',
    'Código':'co_if',
    'TCB':'tp_consolidado_bancario',
    'SR':'segmento',
    'TD':'tp_consolidacao',
    'TC':'tp_controle',
    'Cidade':'cidade',
    'UF':'uf',
    'Data':'dt_base',
    'Disponibilidades (a)':'disponibilidades',
    'Aplicações Interfinanceiras de Liquidez (b)':'aplicacoes_interfinanceiras_liquidez',
    'TVM e Instrumentos Financeiros Derivativos (c)':'tvm_derivativos',
    'Operações de Crédito':'operacoes_credito',    
    'Unnamed: 13':'provisao_operacoes_credito',
    'Unnamed: 14':'operacoes_credito_liquidas_provisao',
    'Arrendamento Mercantil':'arrendamento_mercantil_receber',
    'Unnamed: 16':'imobilizado_de_arrendamento',
    'Unnamed: 17':'credores_antecipacao_valor_residual',
    'Unnamed: 18':'provisao_sobre_arrendamento_mercantil',
    'Unnamed: 19':'arrendamento_mercantil_liquido_provisao',
    'Outros Créditos - Líquido de Provisão (f)':'outros_creditos_liquidos_provisao',
    'Outros Ativos Realizáveis (g)':'outros_ativos_realizaveis',
    'Permanente Ajustado (h)':'permanente_ajustado',
    'Ativo Total Ajustado (i) = (a) + (b) + (c) + (d) + (e) + (f) + (g) + (h)':'ativo_total_ajustado',
    'Credores por Antecipação de Valor Residual (j)':'credores_antecipacao_valor_residual',
    'Ativo Total (k) = (i) - (j)':'ativo_total'
}

# renomeia as colunas
df = df.rename(columns=a_renomear)

# remove informações que são consolidadas
df = df[df['co_if'].notnull()]

# convert just columns "a" and "b"
df['co_if'] = df['co_if'].astype(int)
df['tp_controle'] = df['tp_controle'].astype(int)

print(df.head())
print(df.dtypes)
print(df.tail())

# salva os registros no banco de dados
df.to_sql('{}_import'.format(nome_relatorio), con=engine, if_exists='replace')

# executa para ver os resultados retornados que foram importados
df_banco = engine.execute("SELECT * FROM {}_import".format(nome_relatorio)).fetchall()
print(len(df_banco))

print('Registros importados com sucesso.')

Index(['Unnamed: 0', 'Instituição financeira', 'Código', 'TCB', 'SR', 'TD',
       'TC', 'Cidade', 'UF', 'Data', 'Disponibilidades (a)',
       'Aplicações Interfinanceiras de Liquidez (b)',
       'TVM e Instrumentos Financeiros Derivativos (c)',
       'Operações de Crédito', 'Unnamed: 13', 'Unnamed: 14',
       'Arrendamento Mercantil', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Outros Créditos - Líquido de Provisão (f)',
       'Outros Ativos Realizáveis (g)', 'Permanente Ajustado (h)',
       'Ativo Total Ajustado (i) = (a) + (b) + (c) + (d) + (e) + (f) + (g) + (h)',
       'Credores por Antecipação de Valor Residual (j)',
       'Ativo Total (k) = (i) - (j)', 'Unnamed: 26', 'Conglomerado',
       'Conglomerado Financeiro', 'Conglomerado Prudencial', 'TI',
       'Unnamed: 15', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 28'],
      dtype='object')
                                             nome_if     co_if  \
1                                              

2020-02-12 11:45:08,118 INFO sqlalchemy.engine.base.Engine {}
2020-02-12 11:45:08,129 INFO sqlalchemy.engine.base.Engine COMMIT
2020-02-12 11:45:08,133 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_congl_financeiros_ativo_import_index ON congl_financeiros_ativo_import (index)
2020-02-12 11:45:08,134 INFO sqlalchemy.engine.base.Engine {}
2020-02-12 11:45:08,140 INFO sqlalchemy.engine.base.Engine COMMIT
2020-02-12 11:45:08,415 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-12 11:45:14,655 INFO sqlalchemy.engine.base.Engine INSERT INTO congl_financeiros_ativo_import (index, nome_if, co_if, tp_consolidado_bancario, segmento, tp_consolidacao, tp_controle, cidade, uf, dt_base, disponibilidades, aplicacoes_interfinanceiras_liquidez, tvm_derivativos, operacoes_credito, provisao_operacoes_credito, operacoes_credito_liquidas_provisao, arrendamento_mercantil_receber, imobilizado_de_arrendamento, credores_antecipacao_valor_residual, provisao_sobre_arrendamento_mercantil, arrendam

In [ ]:
nome_relatorio = 'congl_financeiros_passivo'
file_name = '{}.csv'.format(nome_relatorio)
df = pd.read_csv(os.path.join('bases', file_name), low_memory=False)

# remove unnamed columns
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Unnamed: 24', axis=1, inplace=True)
df.drop('Unnamed: 25', axis=1, inplace=True)
df.drop('Unnamed: 32', axis=1, inplace=True)
df.drop('Unnamed: 30', axis=1, inplace=True)

a_renomear = {
    'Instituição financeira':'nome_if',
    'Código':'co_if',
    'TCB':'tp_consolidado_bancario',
    'SR':'segmento',
    'TD':'tp_consolidacao',
    'TC':'tp_controle',
    'Cidade':'cidade',
    'UF':'uf',
    'Data':'dt_base',
    'Captações':'depositos_vista',
    'Unnamed: 10':'depositos_poupanca',
    'Unnamed: 11':'depositos_interfinanceiros',
    'Unnamed: 12':'depositos_prazo',
    'Unnamed: 13':'outros_depositos',
    'Unnamed: 14':'deposito_total',
    'Unnamed: 15':'obrigacoes_operacoes_compromissadas',
    'Unnamed: 16':'letras_credito_imobiliario',
    'Unnamed: 17':'letras_credito_agronegocio',
    'Unnamed: 18':'letras_financeiras',
    'Unnamed: 19':'obrigacoes_titulos_valores_mobiliarios_exterior',
    'Unnamed: 20':'outros_recursos_aceites_emissao_titulos',
    'Unnamed: 21':'recursos_aceites_emissao_titulos',
    'Unnamed: 22':'obrigacoes_emprestimos_repasses',
    'Unnamed: 23':'captacoes',
    'Instrumentos Derivativos (f)':'instrumentos_derivativos',
    'Outras Obrigações (g)':'outras_obrigacoes',
    'Passivo Circulante e Exigível a Longo Prazo (h) = (e) + (f) + (g)':'passivo_circulante_exigivel_longo_prazo',
    'Resultados de Exercícios Futuros (i)':'resultado_exercicios_futuros',
    'Patrimônio Líquido (j)':'patrimonio_liquido',
    'Passivo Total (k) = (h) + (i) + (j)':'passivo_total'
}

# renomeia as colunas
df = df.rename(columns=a_renomear)

# remove informações que são consolidadas
df = df[df['co_if'].notnull()]

# convert just columns "a" and "b"
df['co_if'] = df['co_if'].astype(int)
df['tp_controle'] = df['tp_controle'].astype(int)

print(df.head())
print(df.dtypes)

# salva os registros no banco de dados
df.to_sql('{}_import'.format(nome_relatorio), con=engine, if_exists='replace')

# executa para ver os resultados retornados que foram importados
df_banco = engine.execute("SELECT * FROM {}_import".format(nome_relatorio)).fetchall()
print(len(df_banco))

print('Registros importados com sucesso.')

In [ ]:
nome_relatorio = 'congl_financeiros_demonstracao_resultado'
file_name = '{}.csv'.format(nome_relatorio)
df = pd.read_csv(os.path.join('bases', file_name), low_memory=False)

# remove unnamed columns
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Unnamed: 23', axis=1, inplace=True)
df.drop('Unnamed: 32', axis=1, inplace=True)
df.drop('Unnamed: 33', axis=1, inplace=True)
df.drop('Unnamed: 39', axis=1, inplace=True)
df.drop('Unnamed: 41', axis=1, inplace=True)


a_renomear = {
    'Instituição financeira':'nome_if',
    'Código':'co_if',
    'TCB':'tp_consolidado_bancario',
    'SR':'segmento',
    'TD':'tp_consolidacao',
    'TC':'tp_controle',
    'Cidade':'cidade',
    'UF':'uf',
    'Data':'dt_base',
    'Resultado de Intermediação Financeira':'rendas_op_credito',
    'Unnamed: 10':'rendas_op_arrendamento_mercantil',
    'Unnamed: 11':'Rendas de Operações com TVM (a3)',
    'Unnamed: 12':'rendas_op_derivativos',
    'Unnamed: 13':'resultado_op_cambio',
    'Unnamed: 14':'rendas_aplicacoes_compulsorias',
    'Unnamed: 15':'receitas_intermediacao_financeira',
    'Unnamed: 16':'despesas_captacao',
    'Unnamed: 17':'despesas_obrigacoes_emprestimos_repasses',
    'Unnamed: 18':'despesas_operacoes_arrendamento_mercantil',
    'Unnamed: 19':'resultado_op_cambio',
    'Unnamed: 20':'resultado_provisao_credito_dificil_liquidacao',
    'Unnamed: 21':'despesas_intermediacao_financeira',
    'Unnamed: 22':'resultado_intermediacao_financeira',
    'Outras Receitas/Despesas Operacionais':'rendas_prestacao_servicos',
    'Unnamed: 24':'rendas_tarifas_bancarias',
    'Unnamed: 25':'despesas_pessoal',
    'Unnamed: 26':'despesas_administrativas',
    'Unnamed: 27':'despesas_tributarias',
    'Unnamed: 28':'resultado_participacoes',
    'Unnamed: 29':'outras_receitas_operacionais',
    'Unnamed: 30':'outras_despesas_operacionais',
    'Unnamed: 31':'outras_receitas_despesas_operacionais',
    'Resultado Operacional (e) = (c) + (d)':'resultado_operacional',
    'Resultado Não Operacional (f)':'resultado_nao_operacional',
    'Resultado antes da Tributação, Lucro e Participação (g) = (e) + (f)':'resultado_antes_tributacao_lucro_participacao',
    'Imposto de Renda e Contribuição Social (h)':'imposto_renda_contribuicao_social',
    'Participação nos Lucros (i)':'participacao_lucros',
    'Lucro Líquido (j) = (g) + (h) + (i)':'lucro_liquido',
    'Juros Sobre Capital Próprio (k)':'juros_sobre_capital_proprio',
    'Juros Sobre Capital Social de Cooperativas (k)':'juros_sobre_capital_cooperativas'
}

# renomeia as colunas
df = df.rename(columns=a_renomear)

# remove informações que são consolidadas
df = df[df['co_if'].notnull()]

# convert just columns "a" and "b"
df['co_if'] = df['co_if'].astype(int)
df['tp_controle'] = df['tp_controle'].astype(int)

print(df.head())
print(df.dtypes)

# salva os registros no banco de dados
df.to_sql('{}_import'.format(nome_relatorio), con=engine, if_exists='replace')

# executa para ver os resultados retornados que foram importados
df_banco = engine.execute("SELECT * FROM {}_import".format(nome_relatorio)).fetchall()
print(len(df_banco))

print('Registros importados com sucesso.')